In [1]:
import requests
import gspread
import calendar,time
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import json

In [2]:
GS_CREDENTIAL_FILENAME = "gs_credential10.json"  # İndirilen json dosyası yolu
GS_FILENAME = "User-Req-Sheet"  # Google Sheet dosyası adı
GS_SHEET_INDEX = 0  # Sheet2'e erişim için
gc = gspread.service_account(GS_CREDENTIAL_FILENAME)
wks = gc.open(GS_FILENAME).get_worksheet(GS_SHEET_INDEX)

In [11]:
IntercomUrl = 'https://api.intercom.io/conversations/search'
token= "your_token" #canlı token
admin_search_url = "https://api.intercom.io/admins"
login = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {token}"}

text = ["bugün kargo","müsterim bekliyor",
        "acil","acil dönüş","aciliyet","onay bekliyor",
        "gerekiyor","kargoya verilmemis","sıparısım",'siparisim',
        'kargoya','bugün','hala gelmedi','siparişimin','hızlandırmamız','kargolarını','acil ihtiyacımız var']

#hala, halen, hayla
#İstek atmak istediğimiz link için fonksiyon
def get_data1(url):
    response = requests.post(IntercomUrl, headers = headers,   json=SearchQuery)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(response.text)
        print(response.status_code)

In [4]:
def two_stage(conv_id):
    conv_url = f"https://api.intercom.io/conversations/{conv_id}"
    params = {
    "id":f"{conv_id}"
    }
    login = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response2 = requests.get(url=conv_url,headers=login,json=params)
    data1 = response2.json()
    for i in data1['conversation_parts']['conversation_parts']:
        if i['author']['type'] == 'user':
            if "Siparişim Hazırlanıyor" in i['body']:
                return True
            elif "Siparişim Hazırlanıyor" not in i['body']:
                return False

In [5]:
def three_stage(conv_id):
    conv_url = f"https://api.intercom.io/conversations/{conv_id}"
    params = {
    "id":f"{conv_id}"
    }
    login = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response3 = requests.get(url=conv_url,headers=login,json=params)
    data_f = response3.json()
    j = True
    for m in data_f['conversation_parts']['conversation_parts']:
        if m['author']['type'] == 'user':
            if any(item in (re.sub('<.*?>','',str(m['body']).lower())) for item in text):
                return True
            elif not any(item in (re.sub('<.*?>','',str(m['body']).lower())) for item in text):
                j = False
    return j

In [6]:
def three_stage_print(conv_id):
    conv_url = f"https://api.intercom.io/conversations/{conv_id}"
    params = {
    "id":f"{conv_id}"
    }
    login = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response3 = requests.get(url=conv_url,headers=login,json=params)
    data_f = response3.json()
    for m in data_f['conversation_parts']['conversation_parts']:
        if m['author']['type'] == 'user':
            if any(item in str(m['body']).lower() for item in text):
                body_mess = str(re.sub('<[A-Za-z\/][^>]*>','',str(m['body'])))
                req_date = str(datetime.fromtimestamp(int(m['created_at'])))
                
    return body_mess,req_date

In [7]:
def admin_finder(name_surname):
    response = requests.get(url=admin_search_url, headers=login).json()
    for i in response["admins"]:
        if f"{name_surname}" in i["name"]:
            return i["id"]

In [15]:
def assing_conversation(conv_id, assign_id):
    assing_conversation_url = f"https://api.intercom.io/conversations/{conv_id}/parts"
    assing_params = {
        "type": "admin",
        "admin_id": assign_id,
        "assignee_id": assign_id,
        "message_type": "assignment",
    }
    response = requests.post(url=assing_conversation_url, headers=login, json=assing_params)
    return assign_id

In [17]:
def add_note(conv_id,admin_id):
    add_note_url = f"https://api.intercom.io/conversations/{conv_id}/reply"
    add_note_data = {
        "message_type": "note",
        "type": "admin",
        "admin_id": f"{admin_id}",
        "body": f"{conv_id}'ye not/yorum eklendi.."}
    response = requests.post(url=add_note_url, headers=login, data=add_note_data)

In [7]:
def get_data15():
    now = int(datetime.now().timestamp())
    five = datetime.now() - timedelta(minutes=5)
    five_unix = int(time.mktime(five.timetuple()))
    SearchQuery = {
    "query": {
            "operator": "AND",
            "value": [
                {
                    "field": "created_at",
                    "operator": "<",
                    "value": now
                },
                {
                    "field": "created_at",
                    "operator": ">",
                    "value": five_unix
                }
                
            ]
        }
    }
    response1 = requests.post(IntercomUrl, headers=headers, json=SearchQuery)
    data = response1.json()
    for k in data['conversations']:
        if "Siparişimle ilgili yardım istiyorum" in k['source']['body']:
            conv_id = k['id']
            with open('Bot2.txt',"a+") as txt:
                txt.seek(0)
                lines = [line.rstrip() for line in txt.readlines()]
                if conv_id not in lines:
                    txt.writelines([conv_id + '\n'])
                    x = 0
                    y = 0
                    while x==0:
                        try:
                            two = two_stage(conv_id)
                            if two == True:
                                x+=1
                                pass
                                while y == 0:
                                    three = three_stage(conv_id)
                                    if three == False:
                                        try:
                                            start_range = datetime.now()
                                            dt_string = start_range.strftime("%Y/%m/%d %H:%M:%S")
                                            for l in range(16):
                                                print(f"{l}.inci istek atılıyor,{conv_id}")
                                                three_check = three_stage(conv_id)
                                                time.sleep(30)
                                                if three_check == True:
                                                    a = len(wks.get_values('A:A'))
                                                    wks.update_acell(f"A{a+1}", conv_id) # İlk konuşma conv_id
    
                                                    b = len(wks.get_values('B:B'))
                                                    wks.update_acell(f"B{b+1}", three_stage_print(conv_id)[0]) # Konuşma
                                                    
                                                    c = len(wks.get_values('C:C'))
                                                    wks.update_acell(f"C{c+1}", dt_string) #Beklenmeye başladığı zaman
                                                    three_stage_print(conv_id)
                                                    
                                                    d = len(wks.get_values('D:D'))
                                                    wks.update_acell(f"D{d+1}", three_stage_print(conv_id)[1]) #İstek tarihi
                                                    
                                                    #e = len(wks.get_values('E:E'))
                                                    #wks.update_acell(f"E{e+1}", assing_conversation(conv_id=conv_id,assign_id=admin_finder("Mikail Dogruer"))) #Kime assing edil
                                                    
                                                    #add_note(conv_id, admin_finder("Mikail Dogruer"))
                                                    y+=1
                                                    break
                                                else:
                                                    y+=1
                                                    pass
                                        except:
                                            break
                                    elif three == True:
                                        three_stage_print(conv_id)
                                        y+=1
                                        pass
                            elif two ==False:
                                x+=1
                                pass
                        except:
                            break
                else:
                    txt.close()
                    break
                return True

In [8]:
while True:
    a = get_data15()
    if a == True:
        pass

0.inci istek atılıyor,56911304848771
1.inci istek atılıyor,56911304848771
2.inci istek atılıyor,56911304848771
3.inci istek atılıyor,56911304848771
4.inci istek atılıyor,56911304848771
5.inci istek atılıyor,56911304848771
6.inci istek atılıyor,56911304848771
7.inci istek atılıyor,56911304848771
8.inci istek atılıyor,56911304848771
9.inci istek atılıyor,56911304848771
10.inci istek atılıyor,56911304848771
11.inci istek atılıyor,56911304848771
12.inci istek atılıyor,56911304848771
13.inci istek atılıyor,56911304848771
14.inci istek atılıyor,56911304848771
15.inci istek atılıyor,56911304848771
0.inci istek atılıyor,56911304849175
1.inci istek atılıyor,56911304849175
0.inci istek atılıyor,56911304849380
1.inci istek atılıyor,56911304849380
2.inci istek atılıyor,56911304849380
0.inci istek atılıyor,56911304849931
1.inci istek atılıyor,56911304849931
2.inci istek atılıyor,56911304849931
3.inci istek atılıyor,56911304849931
4.inci istek atılıyor,56911304849931
5.inci istek atılıyor,5691130484


KeyboardInterrupt

